# NLP

In [2]:
from collections import Counter

import os
import string

import copy
import pandas as pd
import pickle
import nltk
from nltk.tokenize import word_tokenize

import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation



from utils import tf_idf
from utils import tree_stem as stem
from utils import text_processing as tp
from bs4 import BeautifulSoup


from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import matplotlib.pyplot as plt

In [17]:
INPUT_DATA_FOLDER = "data"
DATA_FILE = "names.csv"

OUTPUT_FOLDER = "data/preprocessed"
OUTPUT_DATA_FILE = "names_parsed.csv"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "bigram"
count_vectorizer_version = "bigram"

MODEL_FOLDER = "model"

In [3]:
df = pd.read_csv(f"{INPUT_DATA_FOLDER}/{DATA_FILE}")

In [283]:
df = pd.read_csv(f"{OUTPUT_FOLDER}/{OUTPUT_DATA_FILE}", sep=';')


In [ ]:

#df['main_text'] = df['main_html_v2'].apply(lambda x: BeautifulSoup(str(x)).text)

## 0. Check the data

In [ ]:
df.head(5)

## 1. Preprocess text

In [ ]:
df = df.rename(columns={'name': 'text'})

In [ ]:
df

In [ ]:
df['text_preprocessed'] = df['text'].apply(lambda x: tp.preprocess_first(x))

In [ ]:
#df['text_preprocessed_lemm'] = df['text'].apply(lambda x: tp.preprocess_second(x,"lemm"))

In [ ]:
df['text_preprocessed_stemm'] = df['text_preprocessed'].apply(lambda x: tp.preprocess_second(x,"stemm"))

In [4]:
df['text_preprocessed_stemm'] = df['text_preprocessed_stemm'].apply(lambda x: tp.remove_stop_words_second(x))

In [ ]:
# mask = df['text_preprocessed'].str.contains('[a-zA-Z]')
#
# # filter the DataFrame using the mask
# nonenglish_rows = df[~mask]
# nonenglish_rows

In [5]:
df

,Unnamed: 0,id,text,registrationNumber,text_preprocessed,text_preprocessed_stemm
0,0,41797,Проект Закону про внесення змін до деяких зако...,9219,проект закону внесення змін законодавчих акті...,законодавч акт розмежуванн повноваж центральн...
1,1,41791,Проект Закону про внесення змін до деяких зако...,9217,проект закону внесення змін законодавчих акті...,законодавч акт удосконаленн використанн земел...
2,2,41790,Проект Закону про внесення зміни до Кримінальн...,9215,проект закону внесення зміни кримінального ко...,кримінальн кодекс встановленн відповідальн пе...
3,3,41787,Проект Закону про внесення змін до деяких зако...,9213,проект закону внесення змін законів україни в...,відновленн енергетичн безпе зелен трансформац...
4,4,41786,Проект Закону про внесення змін до Закону Укра...,9214,проект закону внесення змін закону україни за...,загальнообов язков державн соціальн страхуван...
...,...,...,...,...,...,...
8672,8672,5,Проект Закону про внесення змін до деяких зако...,2071,проект закону внесення змін законодавчих акті...,законодавч акт державн підтрим сільськ розвит
8673,8673,4,Проект Закону про внесення змін до деяких зако...,2067,проект закону внесення змін законодавчих акті...,законодавч акт успадкуванн прав постійн корис...
8674,8674,3,"Проект Закону про внесення змін до розділу X ""...",1101,проект закону внесення змін розділу перехідні...,розділ перехідн положенн `` земельн кодекс за...
8675,8675,2,Проект Закону громадянами України - засновника...,2065,проект закону громадянами україни засновникам...,громадян засновник фермерськ господарств прав...


In [ ]:
# def stemming(data):
#     stemmer= stem
#     tokens = word_tokenize(str(data))
#     new_text = ""
#     for w in tokens:
#         new_text = new_text + " " + stemmer.stem_word(w)
#         print(stemmer.stem_word(w))
#     return new_text

In [ ]:
df.to_csv(f"{OUTPUT_FOLDER}/{OUTPUT_DATA_FILE}", sep=";", index=False)

In [ ]:
df.to_csv(f"{OUTPUT_FOLDER}/{OUTPUT_DATA_FILE}", sep=";", index=False)



## 2. Merge docs

In [4]:
df = pd.read_csv(f"{OUTPUT_FOLDER}/{OUTPUT_DATA_FILE}", sep=";")

In [5]:
#get the text column 
docs = df['text_preprocessed_stemm'].tolist()

In [6]:
len(docs)

8677

### CountVectorizer

In [7]:
cv = CountVectorizer(ngram_range = (3,3))
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

(8677, 35481)

In [8]:
word_count_vector

<8677x35481 sparse matrix of type '<class 'numpy.int64'>'
	with 133220 stored elements in Compressed Sparse Row format>

In [53]:
#store the content
with open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", 'wb') as handle:
                    pickle.dump(cv, handle)

### TF-IDF

In [54]:

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True,)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [55]:


#store the content
with open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", 'wb') as handle:
                    pickle.dump(tfidf_transformer, handle)

In [56]:
# model_file = f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl"
# with open(model_file, "rb") as f:
#     tfidf_transformer = pickle.load(f)


In [57]:

# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
кодекс адміністративн правопорушенн,4.023541
кримінальн процесуальн кодекс,4.648011
верховн дев ят,4.760279
дев ят скликанн,4.760279
основ законодавч акт,4.811051
...,...
мельник президент увіковіченн,9.375399
мейданович кримінальн провадженн,9.375399
межу російськ федераці,9.375399
майно втрачен пошкоджен,9.375399


In [58]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

<8677x3272 sparse matrix of type '<class 'numpy.float64'>'
	with 124386 stored elements in Compressed Sparse Row format>

In [ ]:
# feature_names = cv.get_feature_names_out()
# # #get tfidf vector for first document
#
# first_document_vector=tf_idf_vector[0]
# #print the scores
# df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
# df.sort_values(by=["tfidf"],ascending=False)

### Test transformtion with fitted models

In [18]:
#load the content
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

In [19]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names_out()

In [20]:
tfidf

TfidfTransformer()

In [21]:
df['keywords_Treegrams'] = df['text_preprocessed_stemm'].apply(lambda x: tf_idf.conver_doc_to_vector(x,cv,tfidf))

In [22]:
df['keywords_Treegrams'].head(5)

0    {'здійсненн державн': 0.335, 'сфер промислов':...
1    {'удосконаленн використанн': 0.597, 'земел обо...
2    {'державн фінансов': 0.406, 'відповідальн пере...
3    {'трансформаці енергетичн': 0.427, 'зелен тран...
4    {'трудов каліцтв': 0.298, 'потерпіл трудов': 0...
Name: keywords_Treegrams, dtype: object

In [ ]:
# test_row = df[df["date"]=="2022-02-27"].iloc[0]
# main_text = test_row["main_text"]
# text_preprocessed = test_row["text_preprocessed"]

# print(page_html_text)
# display(HTML(str(page_html_text)))

In [ ]:
# Завантаження tf-idf векторів
tfidf_vectors = tf_idf_vector

# Створення списку для збереження значень показника кращості кластеризації
silhouette_scores = []
n = 15
# Перебір кількості кластерів від 2 до 20
for n_clusters in range(2, n):
    # Створення об'єкту KMeans з поточною кількістю кластерів
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    # Кластеризація tf-idf векторів
    clusters = kmeans.fit_predict(tfidf_vectors)
    # Розрахунок показника кращості кластеризації (silhouette score)
    silhouette_avg = silhouette_score(tfidf_vectors, clusters)
    silhouette_scores.append(silhouette_avg)

# Візуалізація показників кращості кластеризації
plt.plot(range(2, n), silhouette_scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()

# Вибір кількості кластерів, яка відповідає точці згину на графіку (метод ліктя)
best_n_clusters = np.argmin(np.diff(silhouette_scores)) + 2
print(f'Best number of clusters: {best_n_clusters}')




In [39]:
best_n_clusters = 10

In [ ]:
def sse_scaler_(k_rng , tfidf_matrix):
    sse_scaler  = []
    for k in k_rng:
        km = KMeans(n_clusters=k)
        km.fit(tfidf_matrix)
        km.predict(tfidf_matrix)
        sse_scaler.append(km.inertia_)
    #plot
    plt.plot(k_rng,sse_scaler)
    plt.xlabel("k")
    plt.ylabel("Sum of squared error")
    plt.show()
    plt.close()

In [ ]:
sse_scaler_(np.arange(2, 17) , tfidf_vectors)

In [31]:
lda = LatentDirichletAllocation(n_components=best_n_clusters, max_iter=50, learning_method='online')
topic_word_distribution = lda.fit_transform(tf_idf_vector)
topic_word_distribution


array([[0.01213709, 0.01213709, 0.09549742, ..., 0.01213865, 0.69185755,
        0.01213709],
       [0.02469932, 0.02469932, 0.02469933, ..., 0.4458278 , 0.02469935,
        0.02469929],
       [0.19806274, 0.01944115, 0.01944106, ..., 0.01944182, 0.51431811,
        0.01944106],
       ...,
       [0.01484556, 0.06101023, 0.62762299, ..., 0.01484556, 0.1200012 ,
        0.01484556],
       [0.0179324 , 0.01793241, 0.59634444, ..., 0.14850582, 0.0179324 ,
        0.0179324 ],
       [0.02664148, 0.02664147, 0.0266415 , ..., 0.02664149, 0.20327817,
        0.02664146]])

In [32]:
import numpy as np

# Отримання матриці розподілу тем на слова
topic_word_distribution = lda.components_

# Отримання найпопулярніших слів для кожної теми
n_top_words = 10
top_words = []
for topic_words in topic_word_distribution:
    top_words_indices = topic_words.argsort()[:-n_top_words-1:-1]
    top_words.append([feature_names[i] for i in top_words_indices])

# Виведення найпопулярніших тем і відповідних документів
n_top_topics =8
top_topics_indices = np.argsort(topic_word_distribution.sum(axis=1))[:-n_top_topics-1:-1]
for topic_index in top_topics_indices:
    top_documents_indices = topic_word_distribution[:, topic_index].argsort()[:-n_top_topics-1:-1]
    top_documents = [df.registrationNumber[i] for i in top_documents_indices]
    print("Тема {}: {}".format(topic_index, top_words[topic_index]))
    print("Найпопулярніші документи: {}".format(top_documents))
    print()



Тема 0: ['податков кодекс', 'бюджетн кодекс', 'кримінальн кодекс', 'кримінальн процесуальн', 'процесуальн кодекс', 'прийнятт основ', 'статт кримінальн', 'державн комунальн', 'вищ осві', 'статт бюджетн']
Найпопулярніші документи: ['9203', '9209', '9216', '9218', '9213', '9215', '9214', '9217']

Тема 10: ['загальнообов язков', 'язков державн', 'державн соціальн', 'соціальн страхуванн', 'соціальн захист', 'підвищенн розмір', 'ос інвалідн', 'прийнятт основ', 'державн пенсійн', 'пенсійн страхуванн']
Найпопулярніші документи: ['9203', '9210', '9216', '9218', '9213', '9215', '9214', '9217']

Тема 11: ['кодекс адміністративн', 'адміністративн правопорушенн', 'податков кодекс', 'законодавч акт', 'посиленн відповідальн', 'правопорушенн кримінальн', 'акт забезпеченн', 'кодекс законодавч', 'поверненн доопрацюванн', 'кримінальн кодекс']
Найпопулярніші документи: ['9203', '9213', '0200', '9205', '9216', '9218', '9215', '9214']

Тема 9: ['державн бюджет', 'розділ прикін', 'перехідн положенн', 'прикін

In [ ]:
lda = LatentDirichletAllocation(n_components=3, max_iter=50, learning_method='online')
for i in range(best_n_clusters):
    cluster_docs = tfidf_vectors[kmeans.labels_ == i]
    most_representative_doc = np.argmax(cluster_docs.sum(axis=1))
    #print( i, most_representative_doc)
    cluster_tfidf = cluster_docs[most_representative_doc]
    lda.fit(cluster_tfidf)
    topic_weights = lda.components_
    top_topics = topic_weights.argsort()[::-1][:3]
    print(f"Cluster {i} top topics: {feature_names[top_topics]}")

In [29]:
#c-tf-idf

import numpy as np
import pandas as pd
import scipy.sparse as sp

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


class CTFIDFVectorizer(TfidfTransformer):
    def __init__(self, *args, **kwargs):
        super(CTFIDFVectorizer, self).__init__(*args, **kwargs)

    def fit(self, X: sp.csr_matrix, n_samples: int):
        """Learn the idf vector (global term weights) """
        _, n_features = X.shape
        df = np.squeeze(np.asarray(X.sum(axis=0)))
        idf = np.log(n_samples / df)
        self._idf_diag = sp.diags(idf, offsets=0,
                                  shape=(n_features, n_features),
                                  format='csr',
                                  dtype=np.float64)
        return self

    def transform(self, X: sp.csr_matrix) -> sp.csr_matrix:
        """Transform a count-based matrix to c-TF-IDF """
        X = X * self._idf_diag
        X = normalize(X, axis=1, norm='l1', copy=False)
        return X



In [47]:
#### ПРАЦЮЮЧИЙ КОООД - знаходження найбільш вагомого документу в кластері

representative_docs = []
for i in range(best_n_clusters):
    cluster_docs = tf_idf_vector[kmeans.labels_ == i]
    most_representative_doc = np.array(cluster_docs.mean(axis=1)).flatten().argmax()
    representative_docs.append(most_representative_doc)

# Extract TF-IDF vectors of representative documents
representative_vectors = tf_idf_vector[representative_docs]

# Get top words for each cluster based on TF-IDF weights
top_words_per_cluster = []
for vector in representative_vectors:
    top_indices = vector.toarray()[0].argsort()[:-10:-1]
    top_words = [feature_names[index] for index in top_indices]
    top_words_per_cluster.append(top_words)

# Print the most relevant themes for each cluster
for i, top_words in enumerate(top_words_per_cluster):
    print(f"Cluster {i}: {' | '.join(top_words)}")








Cluster 0: вклад депозит | депозит фізичн | банківськ вклад | оподаткуванн банківськ | акт особлив | особлив оподаткуванн | фізичн ос | кодекс законодавч | податков кодекс
Cluster 1: вчиненн майнов | майнов правопорушен | правопорушен умов | відповідальн вчиненн | кодекс кодекс | воєнн надзвичайн | надзвичайн стан | правопорушенн посиленн | умов воєнн
Cluster 2: основ подоланн | подоланн туберкульоз | прийнятт основ | ґрунт ненавист | здійсню досудов | здійсненн іноземн | здійсню викид | здійсню виробництв | здійсню волонтерськ
Cluster 3: пр цифров | трансформаці федоров | відстав віц | цифров трансформаці | віц пр | здійсню державн | здійсненн транзит | здійсненн функці | здійсненн іноземн
Cluster 4: кори неприбутков | стимулюванн наданн | допомог кори | наданн благодійн | неприбутков організаці | благодійн допомог | кодекс стимулюванн | положенн податков | розділ перехідн
Cluster 5: правопорушенн визначенн | зарядн інфраструктур | єкт зарядн | засоб єкт | визначенн електричн | електр

In [40]:
# Кластеризація з обраною кількістю кластерів
kmeans = KMeans(n_clusters=best_n_clusters, random_state=42)
clusters = kmeans.fit_predict(tf_idf_vector)

# Виведення списку документів для кожного кластеру
for i in range(best_n_clusters):
    print(f'Cluster {i}:')
    print(df["name"][clusters == i].tolist())

Cluster 0:


KeyError: 'act_number'

In [41]:
cl_labels = np.unique(clusters)

In [296]:
df

,Unnamed: 0,act_number,text,text_preprocessed,text_preprocessed_stemm,keywords
0,0,860-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Договору між ...,закон україни ратифікацію договору україною й...,закон україн ратифікаці договор україн йордан...,"{'хашимітськ': 0.447, 'йорданськ': 0.447, 'пра..."
1,1,861-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Договору між ...,закон україни ратифікацію договору україною й...,закон україн ратифікаці договор україн йордан...,"{'хашимітськ': 0.45, 'йорданськ': 0.45, 'корол..."
2,2,880-20,"ЗАКОН УКРАЇНИ\r\nПро ратифікацію Протоколу, що...",закон україни ратифікацію протоколу вносить з...,закон україн ратифікаці протокол вно змін дод...,"{'протокол': 0.727, 'засуджен': 0.296, 'конвен..."
3,3,1035-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Гарантійної у...,закон україни ратифікацію гарантійної угоди у...,закон україн ратифікаці гарантійн угод україн...,"{'підстанці': 0.447, 'надійн': 0.344, 'гаранті..."
4,4,1036-20,ЗАКОН УКРАЇНИ\r\nПро приєднання до Угоди про р...,закон україни приєднання угоди розвиток мульт...,закон україн приєднанн угод розвит мультимода...,"{'мультимодальн': 0.501, 'перевезен': 0.394, '..."
...,...,...,...,...,...,...
2932,0,2339-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Протоколу про...,закон україни ратифікацію протоколу право зас...,закон україн ратифікаці протокол прав застосо...,"{'утриманн': 0.333, 'протокол': 0.316, 'гааз':..."
2933,0,1925-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Протоколу до ...,закон україни ратифікацію протоколу угоди уря...,закон україн ратифікаці протокол угод уряд уг...,"{'уряд': 0.493, 'румун': 0.325, 'словацьк': 0...."
2934,0,2340-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Європейської ...,закон україни ратифікацію європейської конвен...,закон україн ратифікаці європейськ конвенці с...,"{'консульськ': 0.33, 'агент': 0.32, 'легалізац..."
2935,0,2341-20,ЗАКОН УКРАЇНИ\r\nПро ратифікацію Угоди між Каб...,закон україни ратифікацію угоди кабінетом мін...,закон україн ратифікаці угод кабінет міністр ...,"{'угод': 0.345, 'взаємн': 0.326, 'уряд': 0.261..."


In [42]:
docs_clusters = pd.DataFrame({'Document': df['text_preprocessed_stemm'], 'Class': clusters})
docs_per_class = docs_clusters.groupby(['Class'], as_index=False).agg({'Document': ' '.join})

# Create c-TF-IDF
count = CountVectorizer().fit_transform(docs_per_class.Document)
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(docs_clusters))

In [43]:
count_vectorizer = CountVectorizer(ngram_range=(2,2)).fit(docs_per_class.Document)
count = count_vectorizer.transform(docs_per_class.Document)
words = count_vectorizer.get_feature_names_out()
cl_labels = np.arange(docs_clusters.Class.nunique())
# Extract top 10 words per class
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(docs_clusters)).toarray()
words_per_class = {cl_labels[label]: [words[index] for index in ctfidf[label].argsort()[-10:]]
                   for label in docs_per_class.Class}

In [280]:
word = list(words_per_class.items())
word

[(0,
  ['статус',
   'військовослужбов',
   'чорнобильськ',
   'ветеран',
   'гарант',
   'сім',
   'війн',
   'діт',
   'захист',
   'соціальн']),
 (1,
  ['скасуванн',
   'пра',
   'діяльн',
   'ціл',
   'орган',
   'читанн',
   'удосконаленн',
   'сфер',
   'акт',
   'законодавч']),
 (2,
  ['національн',
   'пенсі',
   'соціальн',
   'державн',
   'антикорупційн',
   'бюр',
   'пенсійн',
   'загальнообов',
   'язков',
   'страхуванн']),
 (3,
  ['засоб',
   'вимог',
   'сфер',
   'кримінальн',
   'посиленн',
   'порушенн',
   'кодекс',
   'відповідальн',
   'правопорушенн',
   'адміністративн']),
 (4,
  ['внесенн',
   'бюджетн',
   'ос',
   'забезпеченн',
   'бюджет',
   'діяльн',
   'кодекс',
   'статт',
   'податков',
   'державн']),
 (5,
  ['питан',
   'федераці',
   'слідч',
   'російськ',
   'комісі',
   'район',
   'тимчасов',
   'област',
   'постанов',
   'верховн']),
 (6,
  ['правопорушенн',
   'адміністративн',
   'стан',
   'досудов',
   'посиленн',
   'злочин',
   'відпові

In [320]:
 words = list(words_per_class.items())

In [45]:
joined_dict = {key: '| '.join(value) for key, value in words_per_class.items()}
joined_dict

{0: 'зверненн верховн| направленн депутатськ| депутатськ запит| воєнн стан| соціальн захист| російськ федераці| бюджетн кодекс| народн депутат| державн бюджет| податков кодекс',
 1: 'кодекс встановленн| посиленн відповідальн| кримінальн відповідальн| адміністративн правопорушенн| кодекс адміністративн| статт кримінальн| кодекс кримінальн| кодекс посиленн| правопорушенн кримінальн| кримінальн кодекс',
 2: 'забезпеченн збалансован| кодекс законодавч| реєстр скасуванн| ціл податков| верховн прийнятт| друг читанн| читанн ціл| прийнятт друг| скасуванн рішенн| рішенн верховн',
 3: 'гол секретар| скликанн верховн| обранн гол| секретар член| скликанн поряд| поряд денн| сесі верховн| верховн дев| ят скликанн| дев ят',
 4: 'підрозділ розділ| основ розділ| положенн бюджетн| податков кодекс| пункт розділ| розділ перехідн| положенн податков| розділ прикін| прикін перехідн| перехідн положенн',
 5: 'правопорушенн відповідальн| встановленн відповідальн| правопорушенн встановленн| основ кодекс| посилен

In [ ]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = cl_labels

#plotting the results:

for i in u_labels:
    plt.scatter(docs_clusters.loc[docs_clusters['Class'] == i, 0] , docs_clusters['Class' == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()
